## HW1 Matrix Multiplication
以下我以 markdown 的方式寫註解，解釋各個 cells 的運作內容，做為本次作業的 report。

### Import libraries

In [1]:
from pyspark import SparkConf, SparkContext
import os

### Mapper1
將 input file 根據 ```"\n"``` 切成一行一行，然後把每行轉換成 ```(j, ['M', i, m_ij])``` 或 ```(j, ['N', k, n_jk])``` 的 key-value pair。<br>
Ex1. ```"M,0,2,20"``` → ```(2, ['M', 0, 20])```<br>
Ex2. ```"N,0,1,2"``` → ```(0, ['N', 1, 2])```

In [2]:
def mapper1(lines):
    lines = lines.split("\n")
    maplist = []
    for line in lines:
        items = line.split(",")
        if items[0] == 'M':
            maplist.append((int(items[2]), [items[0], int(items[1]), int(items[3])]))
        elif items[0] == 'N':
            maplist.append((int(items[1]), [items[0], int(items[2]), int(items[3])]))
    return maplist

### Mapper2
將輸入的 key-value pair ```x``` 轉換成 ```[(i, k), m_ij*n_jk]``` 的 key-value pair。<br>
Ex. ```[(0, (['M', 0, 10], ['N', 0, 1]))]``` → ```[(0, 0), 10]```

In [3]:
def mapper2(x):
    out = [(x[1][0][1], x[1][1][1]), x[1][0][2] * x[1][1][2]]
    return out

### Reducer1
將所有擁有相同 key 的 key-value pair 的值相加。

In [4]:
def reducer1(x, y):
    return x+y

### Step1
首先，將檔案 ```500input.txt``` 讀取，經過 mapper1 ，得到第一組 key-value pairs（詳見 function 定義處）。

In [5]:
sc.stop()
conf = SparkConf().setMaster("local").setAppName("matmul")
sc = SparkContext(conf=conf)
pairs = sc.textFile("500input.txt").flatMap(mapper1)
pairs.take(1)

[(0, ['M', 0, 51])]

### Step2
接著使用 ```filter``` 函數，將矩陣 M 的 key-value pairs 跟矩陣 N 的 key-value pairs 分別存進 ```pairs_M``` 與 ```pairs_N``` 裡。

In [6]:
pairs_M = pairs.filter(lambda x: x[1][0] == 'M')
pairs_N = pairs.filter(lambda x: x[1][0] == 'N')
pairs_M.take(1)

[(0, ['M', 0, 51])]

### Step3
使用 ```join``` 函數，將 ```pairs_M``` 與 ```pairs_N``` 中 key 值一樣的 key-value pairs 組合在一起。<br>
Ex. ```[0, ('M', 0, 10)]``` ```[0, ('N', 0, 1)]``` → ```[(0, (['M', 0, 10], ['N', 0, 1]))]```

In [7]:
pairs = pairs_M.join(pairs_N)
pairs.take(1)

[(0, (['M', 0, 51], ['N', 0, 73]))]

### Step4
經過 mapper2，得到第二組 key-value pairs（詳見 function 定義處）。

In [8]:
pairs = pairs.map(mapper2)
pairs.take(1)

[[(0, 0), 3723]]

### Step5
經過 reducer1，將所有擁有相同 key 的 key-value pair 的值相加，再透過 ```sortBy``` 函數，依照 ```i```、```k``` 大小排序。

In [9]:
pairs = pairs.reduceByKey(reducer1).sortBy(lambda x: (x[0][0], x[0][1]))

### Step6
最後，就可以將結果輸出成一份 txt 檔。

In [10]:
f = open("./Outputfile.txt", 'w')
for pair in pairs.collect():
    f.write("%d,%d,%d\n" % (pair[0][0], pair[0][1], pair[1]))

In [11]:
f.close()
sc.stop()